In [1]:
import torch
import pandas as pd
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration

test_df = pd.read_csv("dataset/test_processed.csv")

In [2]:
tokenizer_t5 = T5Tokenizer.from_pretrained("google-t5/t5-large")

device = torch.device("cuda") # GPU usage
model_t5 = T5ForConditionalGeneration.from_pretrained("google-t5/t5-large")
model_t5.to(device)

tokenizer_t5.model_max_length, model_t5

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


(1000000000000000019884624838656,
 T5ForConditionalGeneration(
   (shared): Embedding(32128, 1024)
   (encoder): T5Stack(
     (embed_tokens): Embedding(32128, 1024)
     (block): ModuleList(
       (0): T5Block(
         (layer): ModuleList(
           (0): T5LayerSelfAttention(
             (SelfAttention): T5Attention(
               (q): Linear(in_features=1024, out_features=1024, bias=False)
               (k): Linear(in_features=1024, out_features=1024, bias=False)
               (v): Linear(in_features=1024, out_features=1024, bias=False)
               (o): Linear(in_features=1024, out_features=1024, bias=False)
               (relative_attention_bias): Embedding(32, 16)
             )
             (layer_norm): T5LayerNorm()
             (dropout): Dropout(p=0.1, inplace=False)
           )
           (1): T5LayerFF(
             (DenseReluDense): T5DenseActDense(
               (wi): Linear(in_features=1024, out_features=4096, bias=False)
               (wo): Linear(in_featur

In [3]:
tokenizer_t5.model_max_length = 4096 

In [4]:
def GoBackReadingWindow(model, tokenizer, text):
    torch.cuda.empty_cache()
    if len(text.split()) < tokenizer.model_max_length:
        inputs = tokenizer("summarize: " + text,\
                           return_tensors="pt",\
                           max_length=tokenizer.model_max_length,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=tokenizer.model_max_length)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    OVERLAP = int(tokenizer.model_max_length * 0.2)
    words = text.split()
    chunks = []
    start = 0

    while start < len(words):
        end = min(start + tokenizer.model_max_length, len(words))
        chunks.append(' '.join(words[start:end]))
        start += tokenizer.model_max_length - OVERLAP

    print(len(chunks))
    
    summarized_chunks = []
    for chunk in chunks:
        inputs = tokenizer("summarize: " + chunk,\
                           return_tensors="pt",\
                           max_length=tokenizer.model_max_length,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=tokenizer.model_max_length)
        summarized_chunks.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
        torch.cuda.empty_cache()
    
    summarized_text = ' '.join(summarized_chunks)
    
    if len(summarized_text.split()) > tokenizer.model_max_length:
        return GoBackReadingWindow(model, tokenizer, summarized_text)
    else:
        inputs = tokenizer("summarize: " + summarized_text,\
                           return_tensors="pt",\
                           max_length=tokenizer.model_max_length,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=tokenizer.model_max_length)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [5]:
def ReadingAfterwardsWindow(model, tokenizer, text):
    torch.cuda.empty_cache()
    if len(text.split()) < tokenizer.model_max_length:
        inputs = tokenizer("summarize: " + text,\
                           return_tensors="pt",\
                           max_length=tokenizer.model_max_length,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=tokenizer.model_max_length)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    OVERLAP = int(tokenizer.model_max_length * 0.2)
    words = text.split()
    chunks = []
    start = 0

    isfirts = True
    while start < len(words):
        end = min(start + tokenizer.model_max_length, len(words))
        if isfirts:
            chunks.append(' '.join(words[start:end]))
            isfirts = False
        else:
            chunks.append(' '.join(words[start + OVERLAP:end]) + " " + ' '.join(words[start:start + OVERLAP]))
        start += tokenizer.model_max_length - OVERLAP

    print(len(chunks))
    
    summarized_chunks = []
    for chunk in chunks:
        inputs = tokenizer("summarize: " + chunk,\
                           return_tensors="pt",\
                           max_length=tokenizer.model_max_length,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=tokenizer.model_max_length)
        summarized_chunks.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
        torch.cuda.empty_cache()
    
    summarized_text = ' '.join(summarized_chunks)
    
    if len(summarized_text.split()) > tokenizer.model_max_length:
        return ReadingAfterwardsWindow(model, tokenizer, summarized_text)
    else:
        inputs = tokenizer("summarize: " + summarized_text,\
                           return_tensors="pt",\
                           max_length=tokenizer.model_max_length,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=tokenizer.model_max_length)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [6]:
def ReadingWithJumpingWindow(model, tokenizer, text):
    torch.cuda.empty_cache()
    if len(text.split()) < tokenizer.model_max_length:
        inputs = tokenizer("summarize: " + text,\
                           return_tensors="pt",\
                           max_length=tokenizer.model_max_length,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=tokenizer.model_max_length)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    JUMP = 100
    words = text.split()
    chunks = []
    start = 0

    while start < len(words):
        end = min(int(start + tokenizer.model_max_length + JUMP), len(words))
        chunks.append(' '.join(words[int(start):int(start + tokenizer.model_max_length/2)]) \
                        + " " + ' '.join(words[int(start + tokenizer.model_max_length/2) + JUMP:end]))
        start += tokenizer.model_max_length/2

    print(len(chunks))
    
    summarized_chunks = []
    for chunk in chunks:
        inputs = tokenizer("summarize: " + chunk,\
                           return_tensors="pt",\
                           max_length=tokenizer.model_max_length,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=tokenizer.model_max_length)
        summarized_chunks.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
        torch.cuda.empty_cache()
    
    summarized_text = ' '.join(summarized_chunks)
    
    if len(summarized_text.split()) > tokenizer.model_max_length:
        return ReadingWithJumpingWindow(model, tokenizer, summarized_text)
    else:
        inputs = tokenizer("summarize: " + summarized_text,\
                           return_tensors="pt",\
                           max_length=tokenizer.model_max_length,\
                           truncation=True).to(device)
        outputs = model.generate(**inputs, min_length=0, max_new_tokens=tokenizer.model_max_length)
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

All the tests were performed by using just 30 texts for the following reasons:
- Using all the dataset would take days to elaborate
- The statistical method adopted (kruskal) have as an assumption to have at least 5 observations (https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kruskal.html)

In [7]:
from bert_score import score

F1_t51 = []

for index, row in test_df[:30].iterrows():
    reference_text = row["reference"]
    reference_summary = row["summary"]
    print(row["reference_tokens"])

    result_summary = GoBackReadingWindow(model_t5, tokenizer_t5, reference_text)
    print(result_summary)
    P, R, F1 = score([result_summary], [reference_summary], lang='en', verbose=False)
    print(f"T5 BertScore F1: {F1.item():.2f}")
    F1_t51.append(F1.item())
    torch.cuda.empty_cache()

np.save('F1_t51.npy', F1_t51)

sum1 = 0
for _ in F1_t51:
    sum1 += _
print(sum1 / len(F1_t51))

4996
2
eu regulation establishes uniform rule on publication of national provision concerning marketing requirement for collective investment undertaking. it also provides for the establishment of a central database on the cross-border marketing of collective investment undertaking.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.82
14373
5
eu regulation aims to improve functioning of internal market by strengthening market surveillance of product covered by union harmonisation legislation. each member state shall designate one or more market surveillance authority in its territory.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.81
2073
eu directive aims to enhance cross-border access to a greater number of television and radio programme. it lay down rule for the transmission of television and radio programme through the process of direct injection. member state shall ensure that rightholders may exercise their right to grant or refuse the authorisation for a retransmission.


...

T5 BertScore F1: 0.79
12982
4
the recovery and resilience facility is a new eu instrument. it aims to support member states in achieving milestones and targets. the maximum amount of the loan support for each member state shall not exceed 6,8 % of it 2019 gni.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.81
4734
2
instrument is to be used to support member state in implementing reform. it is necessary to encourage investment and to increase competitiveness. limited part of the work programme, not exceeding 30 % of the yearly allocation, shall be reserved for special measure.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.80
0.8080353379249573


In [8]:
F1_t52 = []

for index, row in test_df[:30].iterrows():
    reference_text = row["reference"]
    reference_summary = row["summary"]
    print(row["reference_tokens"])

    result_summary = ReadingAfterwardsWindow(model_t5, tokenizer_t5, reference_text)
    print(result_summary)
    P, R, F1 = score([result_summary], [reference_summary], lang='en', verbose=False)
    print(f"T5 BertScore F1: {F1.item():.2f}")
    F1_t52.append(F1.item())
    torch.cuda.empty_cache()

np.save('F1_t52.npy', F1_t52)

sum2 = 0
for _ in F1_t52:
    sum2 += _
print(sum2 / len(F1_t52))

4996
2
eu regulation establishes uniform rule on publication of national provision concerning marketing requirement for collective investment undertaking. it also provides for the establishment of a central database on the cross-border marketing of collective investment undertaking.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.82
14373
5
eu regulation aims to improve functioning of internal market by strengthening market surveillance of product covered by union harmonisation legislation. each member state shall designate one or more market surveillance authority in its territory.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.81
2073
eu directive aims to enhance cross-border access to a greater number of television and radio programme. it lay down rule for the transmission of television and radio programme through the process of direct injection. member state shall ensure that rightholders may exercise their right to grant or refuse the authorisation for a retransmission.


...

T5 BertScore F1: 0.78
12982
4
the recovery and resilience facility is a new eu instrument. it aims to support member states in achieving milestones and targets. the facility is to be used exclusively for the benefit of the member state concerned.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.81
4734
2
instrument is to be used to support member state in implementing reform. it is necessary to encourage investment and to increase competitiveness. by 31 december 2030, the commission shall provide a global assessment of the implementation.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.81
0.811858578523


In [9]:
F1_t53 = []

for index, row in test_df[:30].iterrows():
    reference_text = row["reference"]
    reference_summary = row["summary"]
    print(row["reference_tokens"])

    result_summary = ReadingWithJumpingWindow(model_t5, tokenizer_t5, reference_text)
    print(result_summary)
    P, R, F1 = score([result_summary], [reference_summary], lang='en', verbose=False)
    print(f"T5 BertScore F1: {F1.item():.2f}")
    F1_t53.append(F1.item())
    torch.cuda.empty_cache()

np.save('F1_t53.npy', F1_t53)

sum3 = 0
for _ in F1_t53:
    sum3 += _
print(sum3 / len(F1_t53))

4996
3
eu regulation establishes uniform rule on publication of national provision concerning marketing requirement for collective investment undertaking. it also provides for the establishment of a central database on the cross-border marketing of collective investment undertaking.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.82
14373
8
eu regulation aims to improve functioning of internal market by strengthening market surveillance of product covered by union harmonisation legislation. a product subject to legislation referred to in paragraph 5 may be placed on the market only if there is an economic operator established in the union. market surveillance authority shall carry out appropriate check on the characteristic of product on an adequate scale. they shall require economic operator to remove content referring to product from online interface or to require explicit display of warning.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.82
2073
eu directive aims to enhance cross-border access to a greater number of television and radio programme. it lay down rule for the transmission of television and radio programme through the process of direct injection. member state shall ensure that rightholders may exercise their right to grant or refuse the authorisation for a retransmission.


...

T5 BertScore F1: 0.79
12982
7
the recovery and resilience facility is a new eu instrument. it aims to support member states in achieving milestones and targets. the maximum financial contribution shall be calculated for each member state.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.81
4734
3
the instrument is to be used to support member state in implementing reform. it is necessary to encourage investment and to increase competitiveness. by 2025, the commission shall provide to the european parliament and the council a mid-term evaluation report.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


T5 BertScore F1: 0.81
0.8115833659966787
